# Cinemática Diferencial
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/oscar-ramos/fundamentos-robotica-python/blob/main/6-Cinematica-Diferencial/6-1-Cinematica-Diferencial.ipynb)

* Oscar E. Ramos Ponce, Universidad de Ingeniería y Tecnología - UTEC
* Fundamentos de Robótica

Este archivo contiene algunos ejemplos de las diapositivas del curso

In [1]:
import sympy as sp
import numpy as np

* Funciones Auxiliares: por facilidad se importará algunas funciones auxiliares

In [2]:
if 'google.colab' in str(get_ipython()):
    !wget https://raw.githubusercontent.com/oscar-ramos/fundamentos-robotica-python/main/6-Cinematica-Diferencial/utils.py

from utils import srotx, sroty, srotz, sTdh, sVectorFromSkew

### Diap 10: Ejemplo - Velocidad angular

In [3]:
t, dt = sp.symbols("t dt")

# Matriz de rotación
R = sroty(t)

# Derivada con respecto al ángulo t
dRdt = sp.diff(R,t)

# Matriz antisimétrica
w_hat = sp.simplify( dRdt * dt * R.T)
display(w_hat)

Matrix([
[  0, 0, dt],
[  0, 0,  0],
[-dt, 0,  0]])

In [4]:
# Vector de velocidad angular
sVectorFromSkew(w_hat)

Matrix([
[ 0],
[dt],
[ 0]])

### Diap 20: Relación entre velocidad angular y derivadas de roll, pitch, yaw

In [5]:
r, p, y, dr, dp, dy = sp.symbols("r p y dr dp dy")

# Componente de velocidad debido a "roll"
wr = sp.Matrix([0,0,1])*dr

# Componente de velocidad debido a "pitch"
Rz = srotz(r)
wp = Rz[:,1]*dp

# Componente de velocidad debido a "yaw"
Rrp = sp.simplify(srotz(r)*sroty(p))
wy = Rrp[:,0]*dy

# Velocidad angular
w = wr + wp + wy
w

Matrix([
[-dp*sin(r) + dy*cos(p)*cos(r)],
[ dp*cos(r) + dy*sin(r)*cos(p)],
[               dr - dy*sin(p)]])

### Diap 23: Relación entre velocidad angular y derivadas de roll, pitch, yaw

In [6]:
r,p,y,dr,dp,dy = sp.symbols("r p y dr dp dy")

# Matriz de rotación de roll, pitch, yaw
R = sp.simplify(srotz(r)*sroty(p)*srotx(y))

# Derivada temporal de la matriz de rotación (usando la regla de la cadena)
dRdt = sp.simplify(sp.diff(R,r)*dr + sp.diff(R,p)*dp + sp.diff(R,y)*dy)

w_hat = sp.simplify(dRdt * R.T)
w = sVectorFromSkew(w_hat)

print("w_hat: "); display(w_hat)
print("w: "); display(w)

w_hat: 


Matrix([
[                            0,               -dr + dy*sin(p), dp*cos(r) + dy*sin(r)*cos(p)],
[               dr - dy*sin(p),                             0, dp*sin(r) - dy*cos(p)*cos(r)],
[-dp*cos(r) - dy*sin(r)*cos(p), -dp*sin(r) + dy*cos(p)*cos(r),                            0]])

w: 


Matrix([
[-dp*sin(r) + dy*cos(p)*cos(r)],
[ dp*cos(r) + dy*sin(r)*cos(p)],
[               dr - dy*sin(p)]])

### Diap 32: Jacobiano geométrico - Ejemplo 1

In [7]:
q1,q2,l1,l2 = sp.symbols("q1 q2 l1 l2")

# Matrices de transformación homogénea
T01 = sp.Matrix([[sp.cos(q1), -sp.sin(q1), 0, l1*sp.cos(q1)],
                 [sp.sin(q1),  sp.cos(q1), 0, l1*sp.sin(q1)],
                 [         0,           0, 1,             0],
                 [         0,           0, 0,             1]])
T12 = sp.Matrix([[sp.cos(q2), -sp.sin(q2), 0, l2*sp.cos(q2)],
                 [sp.sin(q2),  sp.cos(q2), 0, l2*sp.sin(q2)],
                 [         0,           0, 1,             0],
                 [         0,           0, 0,             1]])
T02 = sp.simplify(T01*T12)

In [8]:
# Ejes z con respecto al sistema 0
z0 = sp.Matrix([[0],[0],[1]]); 
z1 = T01[0:3, 2]

# Puntos con respecto al sistema 0
p0 = sp.Matrix([[0],[0],[0]]); 
p1 = T01[0:3, 3]; 
p2 = T02[0:3, 3]; 

# Componentes de los Jacobianos
Jv1 = z0.cross(p2-p0); 
Jv2 = z1.cross(p2-p1); 
Jw1 = z0; 
Jw2 = z1

In [9]:
# Columnas del Jacobiano geométrico
Jcol1 = sp.Matrix.vstack(Jv1, Jw1)
Jcol2 = sp.Matrix.vstack(Jv2, Jw2)

# Jacobiano geométrico (completo)
J = sp.Matrix.hstack(Jcol1, Jcol2)

print("J:"); display(J)
print("Rango:", sp.Matrix.rank(J))

J:


Matrix([
[-l1*sin(q1) - l2*sin(q1 + q2), -l2*sin(q1 + q2)],
[ l1*cos(q1) + l2*cos(q1 + q2),  l2*cos(q1 + q2)],
[                            0,                0],
[                            0,                0],
[                            0,                0],
[                            1,                1]])

Rango: 2


### Diap 33 y 34: Jacobiano geométrico - Ejemplo 2

In [10]:
q1, q2, q3, q4, L = sp.symbols("q1 q2 q3 q4 L")

# Matrices de transformación homogénea i con respecto a i-1
T01 = sTdh(0, q1, 0, sp.pi/2)
T12 = sTdh(0, q2, 0, sp.pi/2)
T23 = sTdh(L, q3, 0, sp.pi/2)
T34 = sTdh(0, q4, L, 0)

# Matrices de transformación homogénea con respecto a 0
T02 = sp.simplify(T01*T12)
T03 = sp.simplify(T02*T23)
T04 = sp.simplify(T03*T34)

In [11]:
# Ejes z (con respecto al sistema 0)
z0 = sp.Matrix([[0],[0],[1]]);
z1 = T01[0:3, 2]
z2 = T02[0:3, 2]
z3 = T03[0:3, 2]

# Puntos con respecto al sistema 0
p0 = sp.Matrix([0,0,0])
p1 = T01[0:3, 3]
p2 = T02[0:3, 3]
p3 = T03[0:3, 3]
p4 = T04[0:3, 3]

In [12]:
# Componentes del Jacobiano (velocidad lineal)
Jv1 = sp.simplify(z0.cross(p4-p0))       # Artic. revolución
Jv2 = sp.simplify(z1.cross(p4-p1))       # Artic. revolución
Jv3 = sp.simplify(z2.cross(p4-p2))       # Artic. revolución
Jv4 = sp.simplify(z3.cross(p4-p3))       # Artic. revolución
# Componentes del Jacobiano (velocidad angular)
Jw1 = z0          # Artic. revolución
Jw2 = z1          # Artic. revolución 
Jw3 = z2          # Artic. revolución
Jw4 = z3          # Artic. revolución

# Columas del Jacobiano geométrico
Jcol1 = sp.Matrix.vstack(Jv1, Jw1)
Jcol2 = sp.Matrix.vstack(Jv2, Jw2)
Jcol3 = sp.Matrix.vstack(Jv3, Jw3)
Jcol4 = sp.Matrix.vstack(Jv4, Jw4)
# Jacobiano geométrico
J = sp.Matrix.hstack(Jcol1, Jcol2, Jcol3, Jcol4)

In [13]:
print("J1:"); display(Jcol1)
print("J2:"); display(Jcol2)
print("J3:"); display(Jcol3)
print("J4:"); display(Jcol4)

J1:


Matrix([
[-L*((sin(q1)*cos(q2)*cos(q3) - sin(q3)*cos(q1))*cos(q4) + sin(q1)*sin(q2)*sin(q4) + sin(q1)*sin(q2))],
[ L*((sin(q1)*sin(q3) + cos(q1)*cos(q2)*cos(q3))*cos(q4) + sin(q2)*sin(q4)*cos(q1) + sin(q2)*cos(q1))],
[                                                                                                   0],
[                                                                                                   0],
[                                                                                                   0],
[                                                                                                   1]])

J2:


Matrix([
[L*(-sin(q2)*cos(q3)*cos(q4) + sin(q4)*cos(q2) + cos(q2))*cos(q1)],
[L*(-sin(q2)*cos(q3)*cos(q4) + sin(q4)*cos(q2) + cos(q2))*sin(q1)],
[         L*(sin(q2)*sin(q4) + sin(q2) + cos(q2)*cos(q3)*cos(q4))],
[                                                         sin(q1)],
[                                                        -cos(q1)],
[                                                               0]])

J3:


Matrix([
[ L*(sin(q1)*cos(q3) - sin(q3)*cos(q1)*cos(q2))*cos(q4)],
[-L*(sin(q1)*sin(q3)*cos(q2) + cos(q1)*cos(q3))*cos(q4)],
[                            -L*sin(q2)*sin(q3)*cos(q4)],
[                                       sin(q2)*cos(q1)],
[                                       sin(q1)*sin(q2)],
[                                              -cos(q2)]])

J4:


Matrix([
[L*(-sin(q1)*sin(q3)*sin(q4) + sin(q2)*cos(q1)*cos(q4) - sin(q4)*cos(q1)*cos(q2)*cos(q3))],
[ L*(sin(q1)*sin(q2)*cos(q4) - sin(q1)*sin(q4)*cos(q2)*cos(q3) + sin(q3)*sin(q4)*cos(q1))],
[                                          -L*(sin(q2)*sin(q4)*cos(q3) + cos(q2)*cos(q4))],
[                                              -sin(q1)*cos(q3) + sin(q3)*cos(q1)*cos(q2)],
[                                               sin(q1)*sin(q3)*cos(q2) + cos(q1)*cos(q3)],
[                                                                         sin(q2)*sin(q3)]])

In [14]:
# Jacobiano en el punto dado:
Js = J.subs({q1:0, q2:135*sp.pi/180, q3:sp.pi, q4:sp.pi})

print("J(q):"); display(Js)

J(q):


Matrix([
[0, -sqrt(2)*L,         0, -sqrt(2)*L/2],
[0,          0,        -L,            0],
[0,          0,         0, -sqrt(2)*L/2],
[0,          0, sqrt(2)/2,            0],
[0,         -1,         0,           -1],
[1,          0, sqrt(2)/2,            0]])

### Diap 40: Jacobiano analítico - Ejemplo 1

In [15]:
q1, q2, l1, l2 = sp.symbols("q1 q2 l1 l2")

* **Forma 1** (simbólico):

In [16]:
# Cinemática directa (término a término)
x = l1*sp.cos(q1) + l2*sp.cos(q1+q2)
y = l1*sp.sin(q1) + l2*sp.sin(q1+q2)
phi = q1 + q2

# Derivadas (término a término)
dxdq1 = sp.diff(x, q1) 
dxdq2 = sp.diff(x, q2)
dydq1 = sp.diff(y, q1) 
dydq2 = sp.diff(y, q2)
dphidq1 = sp.diff(phi, q1) 
dphidq2 = sp.diff(phi, q2)

# Jacobiano analitico
Ja1 = sp.Matrix([[  dxdq1,   dxdq2],
                [  dydq1,   dydq2],
                [dphidq1, dphidq2]])
display(Ja1)

Matrix([
[-l1*sin(q1) - l2*sin(q1 + q2), -l2*sin(q1 + q2)],
[ l1*cos(q1) + l2*cos(q1 + q2),  l2*cos(q1 + q2)],
[                            1,                1]])

* **Forma 2** (simbólico):

In [17]:
# Cinemática directa (como vector)
X = sp. Matrix([[ l1*sp.cos(q1) + l2*sp.cos(q1+q2)],
                [ l1*sp.sin(q1) + l2*sp.sin(q1+q2)],
                [ q1 + q2]])

# Vector de variables articulares
q = sp.Matrix([q1,q2])

# Jacobiano analítico (usando la función "jacobian")
Ja1 = X.jacobian(q)
display(Ja1)

Matrix([
[-l1*sin(q1) - l2*sin(q1 + q2), -l2*sin(q1 + q2)],
[ l1*cos(q1) + l2*cos(q1 + q2),  l2*cos(q1 + q2)],
[                            1,                1]])

### Diap 42: Jacobiano analítico - Ejemplo 2

In [18]:
q1, q2, q3, d1 = sp.symbols("q1 q2 q3 d1")

# Cinemática directa
X = sp.Matrix([[q3*sp.cos(q2)*sp.cos(q1)],
               [q3*sp.cos(q2)*sp.sin(q1)],
               [d1 + q3*sp.sin(q2)]])

# Variables articulares
q = sp.Matrix([q1, q2, q3])

# Jacobiano analítico
Ja2 = X.jacobian(q)
display(Ja2)

Matrix([
[-q3*sin(q1)*cos(q2), -q3*sin(q2)*cos(q1), cos(q1)*cos(q2)],
[ q3*cos(q1)*cos(q2), -q3*sin(q1)*sin(q2), sin(q1)*cos(q2)],
[                  0,          q3*cos(q2),         sin(q2)]])

### Diap 47: Singularidades Cinemáticas - Ejemplo 1

Se considera solamente la parte correspondiente a la posición

In [19]:
# Extraer solo la parte correspondiente a la posición
Jaxy = Ja1[0:2,0:2]

# Determinante
det = sp.simplify(Jaxy.det())
# Rango
r = Jaxy.rank()

print("El determinante es:"); display(det)
print("El rango es:", r)

El determinante es:


l1*l2*sin(q2)

El rango es: 2


* Caso singular 1: cuando $q_2=0$

In [20]:
J1 = sp.simplify(Jaxy.subs({q2: 0}))
det1 = J1.det()
r1 = J1.rank()

print("Jacobiano analítico cuando q2=0:"); display(J1)
print("Determinante cuando q2=0:", det1)
print("Rango cuando q2=0:", r1)

Jacobiano analítico cuando q2=0:


Matrix([
[(-l1 - l2)*sin(q1), -l2*sin(q1)],
[ (l1 + l2)*cos(q1),  l2*cos(q1)]])

Determinante cuando q2=0: 0
Rango cuando q2=0: 1


* Caso singular 2: cuando $q_2=\pi$

In [21]:
J2 = sp.simplify(Jaxy.subs({q2:sp.pi}))
det2 = J2.det()
r2 = J2.rank()

print("Jacobiano analítico cuando q2=pi:"); display(J2)
print("Determinante cuando q2=pi:", det2)
print("Rango cuando q2=pi:", r2)

Jacobiano analítico cuando q2=pi:


Matrix([
[(-l1 + l2)*sin(q1),  l2*sin(q1)],
[ (l1 - l2)*cos(q1), -l2*cos(q1)]])

Determinante cuando q2=pi: 0
Rango cuando q2=pi: 1


### Diap 48: Singularidades cinemáticas - Ejemplo 2

In [22]:
# Determinante
det = sp.simplify(Ja2.det())
# Rango
r = Ja2.rank()

print("El determinante es:"); display(det)
print("El rango es:", r)

El determinante es:


q3**2*cos(q2)

El rango es: 3


In [23]:
# Singularidad 1: cuando q2=90
J1 = sp.simplify(Ja2.subs({q2:sp.pi/2}))
# Determinante
det1 = J1.det()
# Rango
r1 = J1.rank()

print("Jacobiano analítico cuando q2=pi:"); display(J1)
print("Determinante:", det1)
print("Rango:", r1)

Jacobiano analítico cuando q2=pi:


Matrix([
[0, -q3*cos(q1), 0],
[0, -q3*sin(q1), 0],
[0,           0, 1]])

Determinante: 0
Rango: 2


* Singularidad 2: cuando $q_2 = -90$

In [24]:
J2 = sp.simplify(Ja2.subs({q2:-sp.pi/2}))
det2 = J2.det()
r2 = J2.rank()

print("Jacobiano analítico cuando q2=-pi:"); display(J2)
print("Determinante:", det2)
print("Rango:", r2)

Jacobiano analítico cuando q2=-pi:


Matrix([
[0, q3*cos(q1),  0],
[0, q3*sin(q1),  0],
[0,          0, -1]])

Determinante: 0
Rango: 2


* Singularidad 3: cuando $q_3 = 0$

In [25]:
J3 = sp.simplify(Ja2.subs({q3:0}))
det3 = J3.det()
r3 = J3.rank()

print("Jacobiano analítico cuando q3=0:"); display(J3)
print("Determinante:"); display(det3)
print("Rango:", r3)

Jacobiano analítico cuando q3=0:


Matrix([
[0, 0, cos(q1)*cos(q2)],
[0, 0, sin(q1)*cos(q2)],
[0, 0,         sin(q2)]])

Determinante:


0

Rango: 1


### Diap 51: Cinemática Diferencial Inversa - Ejemplo

In [26]:
q1, q2, q3, L1, L2, L3 = sp.symbols("q1 q2 q3 L1 L2 L3")

# Cinemática directa
x = L1*sp.cos(q1) + L2*sp.cos(q1+q2) + L3*sp.cos(q1+q2+q3)
y = L1*sp.sin(q1) + L2*sp.sin(q1+q2) + L3*sp.sin(q1+q2+q3)
th = q1 + q2 + q3

X = sp.Matrix([x, y, th])
q = sp.Matrix([q1, q2, q3])

# Jacobiano analítico
Ja = X.jacobian(q)
Ja

Matrix([
[-L1*sin(q1) - L2*sin(q1 + q2) - L3*sin(q1 + q2 + q3), -L2*sin(q1 + q2) - L3*sin(q1 + q2 + q3), -L3*sin(q1 + q2 + q3)],
[ L1*cos(q1) + L2*cos(q1 + q2) + L3*cos(q1 + q2 + q3),  L2*cos(q1 + q2) + L3*cos(q1 + q2 + q3),  L3*cos(q1 + q2 + q3)],
[                                                   1,                                       1,                     1]])

In [27]:
# Configuración
qd = [sp.pi/4, -sp.pi/4, -sp.pi/4]

# Reemplazando valores en el Jacobiano
J = Ja.subs({q1: qd[0], q2: qd[1], q3: qd[2], L1:0.5, L2:1, L3:0.5})
J

Matrix([
[              0,     0.25*sqrt(2), 0.25*sqrt(2)],
[0.5*sqrt(2) + 1, 0.25*sqrt(2) + 1, 0.25*sqrt(2)],
[              1,                1,            1]])

In [28]:
# Velocidad deseada
V = sp.Matrix([0.7, 0, 0])

# Velocidad articular
dq = J.inv()*V

print("Velocidad articular deseada:"); display( sp.N(dq, 6) )

Velocidad articular deseada:


Matrix([
[-1.9799],
[ 2.6799],
[   -0.7]])

### Diap 55: Cinemática Diferencial Inversa - Ejemplo

In [29]:
# Jacobiano
J = sp.Matrix([[0, -sp.sqrt(2), 0, -sp.sqrt(2)/2],
               [0, 0, -1, 0],
               [0, 0, 0, -sp.sqrt(2)/2],
               [0, 0, sp.sqrt(2)/2, 0],
               [0, -1, 0, -1],
               [1, 0, sp.sqrt(2)/2, 0]])
J

Matrix([
[0, -sqrt(2),         0, -sqrt(2)/2],
[0,        0,        -1,          0],
[0,        0,         0, -sqrt(2)/2],
[0,        0, sqrt(2)/2,          0],
[0,       -1,         0,         -1],
[1,        0, sqrt(2)/2,          0]])

In [30]:
# Pseudoinversa del Jacobiano (la matriz no es cuadrada)
Jpinv = J.pinv()
Jpinv

Matrix([
[         0, sqrt(2)/3,            0,      -1/3,    0, 1],
[-sqrt(2)/2,         0,    sqrt(2)/2,         0,    0, 0],
[         0,      -2/3,            0, sqrt(2)/3,    0, 0],
[ sqrt(2)/4,         0, -3*sqrt(2)/4,         0, -1/2, 0]])

In [31]:
L = sp.symbols("L")

# Twist deseado
xi = sp.Matrix([0, 0, -L, 0, -sp.sqrt(2)/2, 0])

# Velocidad articular
dq = sp.simplify(Jpinv * xi)
dq

Matrix([
[                  0],
[       -sqrt(2)*L/2],
[                  0],
[sqrt(2)*(3*L + 1)/4]])

**Nota**: si el Jacobiano es más complicado, no utilizar `pinv` sino la expresión explícita de la pseudoinversa:

In [32]:
Jpinv = sp.simplify((J.T*J).inv()*J.T)
Jpinv

Matrix([
[         0, sqrt(2)/3,            0,      -1/3,    0, 1],
[-sqrt(2)/2,         0,    sqrt(2)/2,         0,    0, 0],
[         0,      -2/3,            0, sqrt(2)/3,    0, 0],
[ sqrt(2)/4,         0, -3*sqrt(2)/4,         0, -1/2, 0]])

### Diap 62: Toques y Fuerzas

In [33]:
F = sp.Matrix([1, 0, 0, 0, 0, 0])

tau = J.T * F
tau

Matrix([
[         0],
[  -sqrt(2)],
[         0],
[-sqrt(2)/2]])